# Working with Gemini in Pixeltable

Pixeltable's Gemini integration enables you to access the Gemini LLM via the Google Gemini API.

### Prerequisites

- A Google AI Studio account with an API key (https://aistudio.google.com/app/apikey)

### Important notes

- Google AI Studio usage may incur costs based on your plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

First you'll need to install required libraries and enter a Gemini API key obtained via Google AI Studio.

In [ ]:
%pip install -qU pixeltable google-genai

In [ ]:
import os
import getpass

if 'GEMINI_API_KEY' not in os.environ:
    os.environ['GEMINI_API_KEY'] = getpass.getpass('Google AI Studio API Key:')

Now let's create a Pixeltable directory to hold the tables for our demo.

In [ ]:
import pixeltable as pxt

# Remove the 'gemini_demo' directory and its contents, if it exists
pxt.drop_dir('gemini_demo', force=True)
pxt.create_dir('gemini_demo')

## Generate content

Create a Table: In Pixeltable, create a table with columns to represent your input data and the columns where you want to store the results from Gemini.

In [ ]:
from google.genai.types import GenerateContentConfigDict
from pixeltable.functions import gemini

# Create a table in Pixeltable and pick a model hosted on Google AI Studio with some parameters

t = pxt.create_table('gemini_demo.text', {'input': pxt.String})

config = GenerateContentConfigDict(
    stop_sequences=['\n'],
    max_output_tokens=300,
    temperature=1.0,
    top_p=0.95,
    top_k=40,
)
t.add_computed_column(output=gemini.generate_content(
    t.input,
    model='gemini-2.0-flash',
    config=config
))

In [ ]:
# Ask Gemini to generate some content based on the input
t.insert([
    {'input': 'Write a story about a magic backpack.'},
    {'input': 'Tell me a science joke.'}
])

In [ ]:
# Parse the response into a new column
t.add_computed_column(response=t.output['candidates'][0]['content']['parts'][0]['text'])
t.select(t.input, t.response).head()

## Generate images with Imagen

In [ ]:
from google.genai.types import GenerateImagesConfigDict

images_t = pxt.create_table('gemini_demo.images', {'prompt': pxt.String})

config = GenerateImagesConfigDict(aspect_ratio='16:9')
images_t.add_computed_column(generated_image=gemini.generate_images(
    images_t.prompt,
    model='imagen-4.0-generate-001',
    config=config
))

In [ ]:
images_t.insert([{'prompt': 'A friendly dinosaur playing tennis in a cornfield'}])

In [ ]:
images_t.head()

## Generate video with Veo

In [ ]:
videos_t = pxt.create_table('gemini_demo.videos', {'prompt': pxt.String})

videos_t.add_computed_column(generated_video=gemini.generate_videos(
    videos_t.prompt,
    model='veo-2.0-generate-001',
))

In [ ]:
videos_t.insert([{'prompt': 'A giant pixel floating over the open ocean in a sea of data'}])

In [ ]:
videos_t.head()

## Generate Video from an existing Image

We'll add an additional computed column to our existing `images_t` to animate the generated images.

In [ ]:
images_t.add_computed_column(generated_video=gemini.generate_videos(
    image=images_t.generated_image,
    model='veo-2.0-generate-001',
))

In [ ]:
images_t.head()

### Learn more

To learn more about advanced techniques like RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://docs.pixeltable.com/howto/use-cases/rag-operations) tutorial.

If you have any questions, don't hesitate to reach out.